In [1]:
import pandas as pd
import json
from collections import defaultdict, Counter
import glob
from analysis_func.text_preproc import preproc_text
import numpy as np
from collections import defaultdict, Counter
import glob
import json
from random import sample
import sklearn

import re
import string
import warnings
from bs4 import BeautifulSoup
import pickle as pkl

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('stsb-bert-large')

from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
comments = pd.read_csv("../big_data/qsub_comments.csv", header=0)
submissions = pd.read_csv("../big_data/qsub_posts.csv", header=0)
comments.head()

,Unnamed: 0,author,subreddit,author_flair_text,title,id,parent_id,link_id,created_utc,score
0,0,AndrewIsMyDog,QAnonCasualties,NaN,"I was trying to look that up, thanks. I wasn...",houed9o,t1_houdwfg,t3_ri2h30,1639696241,1
1,1,OrsonPresence,QAnonCasualties,NaN,Here’s a link to the PDF version https://www.i...,houe3w9,t3_ri2cfi,t3_ri2cfi,1639696129,1
2,2,Dralix001,QAnonCasualties,NaN,"&gt;QAnon, Flat Earther (and obviously NASA di...",houdwfg,t3_ri2h30,t3_ri2h30,1639696040,1
3,3,NothingAndNow111,QAnonCasualties,NaN,"Hey, that's great news! I hope he sticks out out.",houdhkq,t3_rglj3x,t3_rglj3x,1639695864,1
4,4,NothingAndNow111,QAnonCasualties,NaN,"I'd be really wary, but... Hopefully this is t...",houd55q,t3_rh8j8y,t3_rh8j8y,1639695715,1


In [3]:
submissions.head()

,Unnamed: 0,author,subreddit,author_flair_text,title,id,parent_id,link_id,created_utc,score
0,0,McNuggetKiller,QAnonCasualties,New User,(Update Post) My mom keeps downplaying it Prev...,ri411r,ri411r,ri411r,1639699767,1
1,1,throwaway4thelurker,QAnonCasualties,NaN,I don't really give a fuck about losing my cou...,ri36e4,ri36e4,ri36e4,1639697208,1
2,2,AndrewIsMyDog,QAnonCasualties,NaN,Do most QAnon people believe in multiple consp...,ri2h30,ri2h30,ri2h30,1639695142,1
3,3,OrsonPresence,QAnonCasualties,NaN,"QAnon is evil Reading this, it struck me how t...",ri2cfi,ri2cfi,ri2cfi,1639694747,1
4,4,JustAcivilian24,QAnonCasualties,NaN,Seeing Anti vax relatives for Xmas Not sure if...,ri0u9m,ri0u9m,ri0u9m,1639690465,1


In [4]:
full_data = pd.concat([comments, submissions])
print(len(full_data))

557271


In [6]:
full_data.head()

,Unnamed: 0,author,subreddit,author_flair_text,title,id,parent_id,link_id,created_utc,score
0,0,AndrewIsMyDog,QAnonCasualties,NaN,"I was trying to look that up, thanks. I wasn...",houed9o,t1_houdwfg,t3_ri2h30,1639696241,1
1,1,OrsonPresence,QAnonCasualties,NaN,Here’s a link to the PDF version https://www.i...,houe3w9,t3_ri2cfi,t3_ri2cfi,1639696129,1
2,2,Dralix001,QAnonCasualties,NaN,"&gt;QAnon, Flat Earther (and obviously NASA di...",houdwfg,t3_ri2h30,t3_ri2h30,1639696040,1
3,3,NothingAndNow111,QAnonCasualties,NaN,"Hey, that's great news! I hope he sticks out out.",houdhkq,t3_rglj3x,t3_rglj3x,1639695864,1
4,4,NothingAndNow111,QAnonCasualties,NaN,"I'd be really wary, but... Hopefully this is t...",houd55q,t3_rh8j8y,t3_rh8j8y,1639695715,1


In [7]:
full_data.to_csv("../big_data/full_data.csv")